In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models


In [3]:
# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels to match VGG input
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download the MNIST data
train_data = datasets.MNIST(root='./data_altered', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data_altered', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:03<00:00, 3017711.70it/s]


Extracting ./data_altered/MNIST/raw/train-images-idx3-ubyte.gz to ./data_altered/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 109228.28it/s]


Extracting ./data_altered/MNIST/raw/train-labels-idx1-ubyte.gz to ./data_altered/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 951313.26it/s]


Extracting ./data_altered/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data_altered/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 1372912.13it/s]


Extracting ./data_altered/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data_altered/MNIST/raw



In [4]:
# Load the pre-trained VGG16 model
vgg = models.vgg16(pretrained=True)

# Freeze the parameters of the pre-trained model
for param in vgg.parameters():
    param.requires_grad = False

# Modify the classifier to add more layers
vgg.classifier = nn.Sequential(
    nn.Linear(vgg.classifier[0].in_features, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 2048),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 10)
)

net = vgg
print(net)


/home/arunbh/.pyenv/versions/3.10.14/envs/env310/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/arunbh/.pyenv/versions/3.10.14/envs/env310/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/arunbh/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 528M/

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
trainloader = DataLoader(train_data, batch_size=32, shuffle=True)
testloader = DataLoader(test_data, batch_size=32, shuffle=False)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.classifier.parameters(), lr=0.001)


In [ ]:
for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

Epoch 1, Batch 200, Loss: 0.397
Epoch 1, Batch 400, Loss: 0.185
Epoch 1, Batch 600, Loss: 0.184
Epoch 1, Batch 800, Loss: 0.196
Epoch 1, Batch 1000, Loss: 0.178
Epoch 1, Batch 1200, Loss: 0.175
Epoch 1, Batch 1400, Loss: 0.143
Epoch 1, Batch 1600, Loss: 0.168
Epoch 1, Batch 1800, Loss: 0.186
Epoch 2, Batch 200, Loss: 0.096
Epoch 2, Batch 400, Loss: 0.129
Epoch 2, Batch 600, Loss: 0.154
Epoch 2, Batch 800, Loss: 0.112
Epoch 2, Batch 1000, Loss: 0.132
Epoch 2, Batch 1200, Loss: 0.139
Epoch 2, Batch 1400, Loss: 0.149
Epoch 2, Batch 1600, Loss: 0.123
Epoch 2, Batch 1800, Loss: 0.141
Epoch 3, Batch 200, Loss: 0.106


In [16]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test set: {100 * correct / total:.2f}%')


Accuracy of the network on the test set: 97.74%


In [ ]:
# Save the model weights
torch.save(net.state_dict(), 'vgg_mnist_weights.pth')
print("Model weights saved successfully")

In [ ]:
# Load the model weights
net.load_state_dict(torch.load('vgg_mnist_weights.pth'))
net.eval()  # Set the model to evaluation mode
print("Model weights loaded successfully")

# Evaluate the loaded model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the loaded network on the test set: {100 * correct / total:.2f}%')
